#  Szybka eksploracja danych i budowanie modeli predykcyjnych z użyciem Pythona

#### Autor: Mikołaj Sędek, Data Scientist, Grupa Pracuj Sp. z.o.o.

##### GIT: https://github.com/MikolajSedek/PythonCode/tree/master/DataScienceWBiznesie_7_12_2016

<img src="img/img005.png", width=400,height=400>
<img src="img/img001.png", width=400,height=400>
<img src="img/img002.png", width=400,height=400>
<img src="img/img003.png", width=400,height=400>
<img src="img/img004.jpg", width=400,height=400>


### Załadujmy moduły potrzebne nam do pracy

##### Uwaga: instalacja modułów xgboost i keras/theano w środowisku Windows jest czasochłonna i skomplikowana, w Linuxie jest o wiele łatwiej

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
#import modułów potrzebnych do modelowania

import xgboost
import sklearn
import pandas as pd
import keras
import seaborn as sns
import matplotlib
import imblearn
import numpy as np
import matplotlib.pyplot as plt
import numpy
import os
import h5py
import theano

from imblearn.under_sampling import RandomUnderSampler
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegressionCV

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import FunctionTransformer


from xgboost import XGBClassifier

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.regularizers import l1l2


%matplotlib inline


## Takie wersje modułów są zainstalowane na mojej maszynie:
xgboost:0.4
sklearn:0.18
pandas:0.19.1
keras:1.1.0
theano: 0.8.2
seaborn:0.7.1
matplotlib:1.5.1
imblearn:0.1.8
h5py:2.6.0

In [ ]:
#sprawdzenie wersji modułów

print("xgboost:"+str(xgboost.__version__))
print("sklearn:"+str(sklearn.__version__))
print("pandas:"+str(pd.__version__))
print("keras:"+str(keras.__version__))
print("theano:"+str(theano.__version__))
print("seaborn:"+str(sns.__version__))
print("matplotlib:"+str(matplotlib.__version__))
print("imblearn:"+str(imblearn.__version__))
print("h5py:"+str(h5py.__version__))


### Pobranie danych zajmie ok. 30 sekund - to ponad 300 tys. rekordów

In [ ]:
#ładujemy dane z plików csv

dataset_learn = pd.read_csv("datasets\data_learning.csv", sep=";", decimal=".", compression="gzip",index_col=False)

dataset_apply = pd.read_csv("datasets\data_apply.csv", sep=";", decimal=".", compression="gzip",index_col=False)


In [ ]:
#Zobaczmy z jakimi danymi mamy do czynienia

print("dane uczące - wiersze,kolumny: " + str(dataset_learn.shape))

print("dane wdrożeniowe - wiersze,kolumny: " + str(dataset_apply.shape))



In [ ]:
#zmienne w zbiorze uczącym i wdrożeniowym

dl_vars= list(dataset_learn.columns.values)
da_vars= list(dataset_apply.columns.values)


print(dl_vars)


In [ ]:
print(da_vars)


### Zobaczmy jak wygląda ramka danych na których będziemy pracować

In [ ]:
dataset_learn.iloc[0:10,0:10]

In [ ]:
dataset_learn.iloc[0:10,380:390]

### Sprawdźmy częstości w zmiennej celu - może się okazać że będzie trzeba będzie zbalansować dane uczące

In [ ]:
dataset_learn.groupby("Target").size().plot(kind="bar",
                                            title="Zmienna celu - zliczenia", colormap = "coolwarm")

In [ ]:
print(dataset_learn.groupby("Target").size())

### Wiele modeli preferuje zmienną celu w formie binarnej - 0/1

In [ ]:
#przekodowanie zmiennej celu na zmienną binarną

#skorzystamy z fukncji dostępnej w sklearn
le = LabelEncoder()
le.fit(dataset_learn["Target"])

print("oryginalne klasy w zmiennej celu: " +str(le.classes_))
Target_New = le.transform(dataset_learn["Target"])

dataset_learn["Target"]=Target_New
dataset_learn.groupby("Target").size()



### Zadbajmy o braki danych oraz usunięcie zmiennych z zerową wariancją

In [ ]:
#usuwamy braki danych

#znajdźmy zmienne z brakami danych
dict_miss = {k:v for (k,v) in dict(dataset_learn.isnull().sum()).items() if v > 1}

pd.DataFrame.from_dict(dict_miss,orient="index").plot(kind="bar",legend=False,
                                                      title="Zmienne z brakami danych",
                                                      ylim=(0,dataset_learn.shape[0]),
                                                     colormap="coolwarm")


In [ ]:
#wypełnienie braków danych medianą
miss_vars = dict_miss.keys()

dataset_learn[miss_vars] = dataset_learn[miss_vars].fillna(dataset_learn[miss_vars].median())

dict_miss_check = {k:v for (k,v) in dict(dataset_learn.isnull().sum()).items() if v > 1}

print("lista zmiennych w których nadal występują braki danych: "+ str(dict_miss_check))


In [ ]:
#usuwamy zmienne mające zerową wariancję (stałych :) )
  

vars_unique = pd.DataFrame(dataset_learn.apply(pd.Series.nunique))

vars_constant = list(vars_unique[vars_unique[0]==1].index)

print("zmienne w których występuje zerowa wariancja:")
print(str(vars_constant))

print(dataset_learn[vars_constant].head)


dataset_learn.drop(vars_constant,axis=1,inplace=True)


### Podzielmy nasze dane na zbiór uczący i testowy, zbalansujmy dane uczące

In [ ]:
#podział na zbiór testowy i uczący

seed = 7

training, testing = train_test_split(dataset_learn, test_size = 0.2, random_state=seed)

print("wielkość zbioru uczącego :"+ str(training.shape))
print("wielkość zbioru testowego :"+ str(testing.shape))


#### Balansowanie zbioru uczącego można wykonać za pomocą modułu imblern 

dla osób zainteresowanych tematyką niezbalansowanych zbiorów uczących:<br>
http://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/ <br>
https://www.quora.com/In-classification-how-do-you-handle-an-unbalanced-training-set <br>
https://github.com/scikit-learn-contrib/imbalanced-learn <br>


In [ ]:
#zbalansowanie zbioru uczącego

iv_vars = list(training.columns.values)
iv_vars.remove("Target")

#definicja metody próbkowania
rus = RandomUnderSampler(replacement=False, random_state=seed)

training_rs_ivs, training_rs_dv = rus.fit_sample(training[iv_vars], training["Target"])



In [ ]:
print("liczba zer: "+str(list(training_rs_dv).count(0)))
print("liczba jedynek: "+str(list(training_rs_dv).count(1)))


In [ ]:
#tworzymy nową ramkę danych ze zbalansowanymi danymi treningowymi

training_bal = pd.DataFrame(training_rs_ivs)
training_bal.columns = iv_vars
training_bal["Target"]=training_rs_dv
training_bal=training_bal.sample(frac=1)

training_bal.iloc[0:10,0:10]

training_bal.groupby("Target").size().plot(kind="bar",
                                           title="Zbalansowana zmienna celu w nowej ramce danych",
                                          colormap="coolwarm")


### Bardzo ważnym etapem procesu jest dobór optymalnych predyktorów
#### Ponieważ korzystamy z anonimowych zmiennych zastosujemy statystykę ważności zmiennych z modelu Random Forest (analiza wrażliwości)
#### http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html


In [ ]:
#dobór zmiennych z użyciem Random Forest 


rf = RandomForestClassifier(n_estimators=200, criterion='gini', max_depth=10, min_samples_split=20, min_samples_leaf=10, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_split=1e-07, bootstrap=True, oob_score=True, n_jobs=-1, random_state=seed, verbose=1, warm_start=False, class_weight=None)

rf.fit(training_bal[iv_vars],training_bal["Target"])



In [ ]:
#selekcja najlepszych zmiennych dzięki statytyskom ważności z Random Forest

features_frame = pd.DataFrame()

features_frame["vars"]=iv_vars
features_frame["importance"]=rf.feature_importances_

features_frame.sort_values(by="importance",ascending=False,inplace=True)

features_frame[features_frame.importance>0].plot(kind="area",use_index=False, colormap="coolwarm")

features_frame.iloc[0:30].plot(kind="bar",x="vars",y="importance", colormap="coolwarm")




In [ ]:
print("zmienne o ważności większej niż 1%: "+ str(features_frame[features_frame.importance>0.01].shape[0]))

imp_vars = list(features_frame[features_frame.importance>0.01].vars)

print(imp_vars)

In [ ]:
#zbadajmy przy okazji korelacje między predyktorami - np. 5 najważniejszymi wg rankingu Random Forest
#przy okazji może odryjemy coś ciekawego związanego z rozkładami zmiennych


top5vars = list(features_frame.vars[0:5])

sns.set(style="ticks", color_codes=True)

sns.pairplot(training_bal[top5vars].sample(frac=0.5))





In [ ]:
#może zbadamy relacje między dwiema zmiennymi bardziej dokładnie - na próbce danych

sns.jointplot("v110", "v17", data=training_bal.sample(frac=0.5), kind="reg")




# Rozpoczynamy budowanie modeli predykcyjnych

Na początek klasyczny model statystyczny - regresja logistyczna, zobaczymy jak poradzi sobie z naszymi danymi. Potem zastosujemy 
dużo potężniejsze uczenie maszynowe :).


In [ ]:
#będziemy chcieli "poprawić" dystrybucje zmiennych dzięki przekształceniu logarytmicznemu log(1+wartość zmiennej),
#upewnijmy się że w naszych zmiennych nie ma wartości ujemnych

training_bal_neg = training_bal.columns[(training_bal < 0).any()].tolist()
testing_neg = testing.columns[(testing < 0).any()].tolist()

if len(training_bal_neg)>0:
    for i in training_bal_neg:
        training_bal[i]=np.where(training_bal[i]<0,0,training_bal[i])

if len(testing_neg)>0:
    for i in testing_neg:
        testing[i]=np.where(testing[i]<0,0,testing[i])

print("zmienne z wartościami ujemnymi w zbiorze uczącym: "+ str(training_bal_neg))        
print("zmienne z wartościami ujemnymi w zbiorze testowym: "+ str(testing_neg))        
        


### Stworzymy sobie własną funkcję transformującą - skorzystamy z funkcji numpy log1p i przetestujemy ją na danych
https://docs.scipy.org/doc/numpy/reference/generated/numpy.log1p.html <br>
http://scikit-learn.org/stable/modules/preprocessing.html#function-transformer <br>

In [ ]:

transformer_log1p = FunctionTransformer(np.log1p)

sns.pairplot(training_bal[top5vars].apply(np.log1p, axis=0).sample(frac=0.2))



In [ ]:
#regresja logistyczna z kroswalidacją i regularyzacją typu l2 (ridge regression)
lr = LogisticRegressionCV(Cs=10, fit_intercept=True, cv=5, dual=False, penalty='l2', scoring=None, solver='lbfgs', tol=0.0001, max_iter=100, class_weight=None, n_jobs=-1, verbose=1, refit=True, intercept_scaling=1.0, multi_class='ovr', random_state=seed)


estimators = []
estimators.append(('log1p', transformer_log1p))
estimators.append(('lr_model', lr))
pipeline_lr = Pipeline(estimators)


pipeline_lr.fit(training_bal[imp_vars],training_bal["Target"])



In [ ]:
#sprawdźmy jakość naszego modelu na danych testowych

pred_lr= pipeline_lr.predict(testing[imp_vars])
pred_lr_proba = pipeline_lr.predict_proba(testing[imp_vars])

pred_lr_proba_1 = list(pd.DataFrame(pred_lr_proba)[1])


In [ ]:
#liczymy macierz błędu dla regresji logistycznej

cf_lr=confusion_matrix(testing["Target"],pred_lr)

cf_lr_pd = pd.DataFrame(cf_lr)
cf_lr_pd.columns=["predykcja 0","predykcja 1"]
cf_lr_pd.index = ["realne 0","realne 1"]


cf_lr_pd.plot(kind="bar",title="predykcja konwersji wg modelu regresji logistycznej")

sens = round(cf_lr[1][1]*1.00 / np.sum(cf_lr[1]),2)
spec = round(cf_lr[0][0]*1.00 / np.sum(cf_lr[0]),2)
aucc = round(roc_auc_score(y_true=testing["Target"],y_score=pred_lr_proba_1),2)

print("czułość: "+str(sens))
print("specyficzność: "+str(spec))
print("powierzchnia pod krzywą:"+str(aucc))



In [ ]:
#spróbujmy to narysować :)

fpr_lr, tpr_lr, thresholds_lr = roc_curve(testing["Target"], pred_lr_proba_1, pos_label=1, drop_intermediate = False)
roc_auc_lr = auc(fpr_lr, tpr_lr)

plt.figure()
lw = 2
plt.plot(fpr_lr, tpr_lr, color='darkgreen',
         lw=lw, label=u'Krzywa ROC reg.logistyczna (obszar pod krzywą = %0.2f)' % roc_auc_lr)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel(u'1 - Specyficzność')
plt.ylabel(u'Czułość')
plt.title('Krzywa ROC dla regresji logistycznej')
plt.legend(loc="lower right")
plt.show()



## Czas na ciężką artylerię - Xtreeme Gradient Boosting Trees 

### https://xgboost.readthedocs.io/en/latest/

In [ ]:
model_xgbm = XGBClassifier(base_score=0.5, colsample_bylevel=0.5, colsample_bytree=0.5,
       gamma=0, learning_rate=0.02, max_delta_step=0, max_depth=4,
       min_child_weight=1, missing=None, n_estimators=2000, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1e-4,
       scale_pos_weight=1, seed=seed, silent=False, subsample=0.5)

#przygotowujemy dane do uczenia i testowania w formie macierzy - będzie szybciej :)
training_X = training_bal[imp_vars].apply(np.log1p, axis=0).as_matrix()
training_y = training_bal["Target"].as_matrix()

testing_X= testing[imp_vars].apply(np.log1p, axis=0).as_matrix()
testing_y = testing["Target"].as_matrix()


model_xgbm.fit(training_X,training_y, early_stopping_rounds=20, eval_metric="auc", eval_set=[(testing_X, testing_y)])





In [ ]:
#sprawdźmy jakość naszego modelu na danych testowych

pred_xgb= model_xgbm.predict(testing_X)
pred_xgb_proba = model_xgbm.predict_proba(testing_X)

pred_xgb_proba_1 = list(pd.DataFrame(pred_xgb_proba)[1])


In [ ]:
#liczymy macierz błędu dla xgboost

cf_xgb=confusion_matrix(testing["Target"],pred_xgb)

cf_xgb_pd = pd.DataFrame(cf_xgb)
cf_xgb_pd.columns=["predykcja 0","predykcja 1"]
cf_xgb_pd.index = ["realne 0","realne 1"]


cf_xgb_pd.plot(kind="bar",title="predykcja konwersji wg modelu xgboost")


sens = round(cf_xgb[1][1]*1.00 / np.sum(cf_xgb[1]),2)
spec = round(cf_xgb[0][0]*1.00 / np.sum(cf_xgb[0]),2)
aucc = round(roc_auc_score(y_true=testing["Target"],y_score=pred_xgb_proba_1),2)

print("czułość: "+str(sens))
print("specyficzność: "+str(spec))
print("powierzchnia pod krzywą:"+str(aucc))



In [ ]:
fpr_xgb, tpr_xgb, thresholds_xgb = roc_curve(testing["Target"], pred_xgb_proba_1, pos_label=1, drop_intermediate = False)

roc_auc_xgb = auc(fpr_xgb, tpr_xgb)


In [ ]:

plt.figure()
lw = 2
plt.plot(fpr_lr, tpr_lr, color='darkgreen',
         lw=lw, label=u'Krzywa ROC reg.logistyczna (obszar pod krzywą = %0.2f)' % roc_auc_lr)
plt.plot(fpr_xgb, tpr_xgb, color='red',
         lw=lw, label=u'Krzywa ROC xgboost (obszar pod krzywą = %0.2f)' % roc_auc_xgb, linestyle='--')

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel(u'1 - Specyficzność')
plt.ylabel(u'Czułość')
plt.title('Krzywa ROC dla wielu modeli')
plt.legend(loc="lower right")
plt.show()


## Spróbujmy jeszcze z Support Vector Machines

### http://scikit-learn.org/stable/modules/svm.html


In [ ]:
training_svm = training_bal.sample(n=10000,replace=False)

from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC

#konfigurujemy model z użyciem baggingu
n_estimators = 10
model_svm = BaggingClassifier(SVC(kernel="linear", C=3, gamma='auto', max_iter=-1, probability=True, random_state=seed, shrinking=True, verbose=True, tol=1e-5), max_samples=1.0 / n_estimators, n_estimators=n_estimators, oob_score=True, random_state=seed)

#dodajmy standaryzację - model svm tak jak sieć neuronowa może być na to wrażliwy
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('svm', model_svm))
pipeline_svm = Pipeline(estimators)



#calculate model
pipeline_svm.fit(training_svm[imp_vars],training_svm["Target"])


In [ ]:
#sprawdźmy jakość naszego modelu na danych testowych

pred_svm= pipeline_svm.predict(testing[imp_vars])
pred_svm_proba = pipeline_svm.predict_proba(testing[imp_vars])

pred_svm_proba_1 = list(pd.DataFrame(pred_svm_proba)[1])

#liczymy macierz błędu dla svm

cf_svm=confusion_matrix(testing["Target"],pred_svm)

cf_svm_pd = pd.DataFrame(cf_svm)
cf_svm_pd.columns=["predykcja 0","predykcja 1"]
cf_svm_pd.index = ["realne 0","realne 1"]
cf_svm_pd.plot(kind="bar",title="predykcja konwersji wg modelu svm")

sens = round(cf_svm[1][1]*1.00 / np.sum(cf_svm[1]),2)
spec = round(cf_svm[0][0]*1.00 / np.sum(cf_svm[0]),2)
aucc = round(roc_auc_score(y_true=testing["Target"],y_score=pred_svm_proba_1),2)

print("czułość: "+str(sens))
print("specyficzność: "+str(spec))
print("powierzchnia pod krzywą:"+str(aucc))


In [ ]:
fpr_svm, tpr_svm, thresholds_svm = roc_curve(testing["Target"], pred_svm_proba_1, pos_label=1, drop_intermediate = False)

roc_auc_svm = auc(fpr_svm, tpr_svm)

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr_lr, tpr_lr, color='darkgreen',
         lw=lw, label=u'Krzywa ROC reg.logistyczna (obszar pod krzywą = %0.2f)' % roc_auc_lr)
plt.plot(fpr_xgb, tpr_xgb, color='red',
         lw=lw, label=u'Krzywa ROC xgboost (obszar pod krzywą = %0.2f)' % roc_auc_xgb, linestyle='--')
plt.plot(fpr_svm, tpr_svm, color='darkviolet',
         lw=lw, label=u'Krzywa ROC svm bagging (obszar pod krzywą = %0.2f)' % roc_auc_svm, linestyle=':')

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel(u'1 - Specyficzność')
plt.ylabel(u'Czułość')
plt.title('Krzywe ROC dla wielu modeli')
plt.legend(loc="lower right")
plt.show()


## A teraz spróbujmy podejścia z sieciami neuronowymi

### https://keras.io/


In [ ]:
#dajmy kerasowi pełną moc na CPU 

#multithreading
import multiprocessing
num_cores = str(multiprocessing.cpu_count())

import os
os.environ['OMP_NUM_THREADS'] = num_cores
os.environ['GOTO_NUM_THREADS'] = num_cores
os.environ['OPENBLAS_NUM_THREADS'] = num_cores

import mkl
mkl.set_num_threads(int(num_cores))


In [ ]:
#skorzystamy z wrappera KerasClassifier który pozwoli nam skorzystać z modelu Keras/Theano w ScikitLearn

def create_model():
    # create model
    model = Sequential()
    model.add(Dropout(0.2, input_shape=(len(imp_vars),)))
    model.add(Dense(100, init='normal', activation='relu', W_constraint=maxnorm(2), W_regularizer=l1l2(l1=0,l2=1e-4)))
    model.add(Dropout(0.2))
    model.add(Dense(60, init='normal', activation='relu', W_constraint=maxnorm(2), W_regularizer=l1l2(l1=0,l2=1e-4)))
    model.add(Dropout(0.2))
    model.add(Dense(1, init='normal', activation='sigmoid'))
    # Compile model
    sgd = SGD(lr=0.005, momentum=0.9, decay=0.0, nesterov=True)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['fbeta_score'])
    return model


In [ ]:
#dodajemy możliwość zatrzymania nauki modelu gdy przez 5 kolejnych epok wyniki się nie poprawią - na podstawie statystyk z danych walidacyjnych
earlyStopping=keras.callbacks.EarlyStopping(monitor='val_fbeta_score', patience=5, verbose=1, mode='auto')

#zapamiętujemy najlepszy model w pliku - uwaga! konieczna instalacja biblioteki h5py
filepath="temp\model_best.hdf5"
modelCheck = keras.callbacks.ModelCheckpoint(filepath, monitor='val_fbeta_score', verbose=1, save_best_only=True, mode='auto')


#przygotowujemy dane do uczenia i testowania w formie macierzy - będzie szybciej :)
training_X = training_bal[imp_vars].as_matrix()
training_y = training_bal["Target"]

testing_X= testing[imp_vars].as_matrix()
testing_y = testing["Target"].as_matrix()

seed = 7
numpy.random.seed(seed)
estimators = []
estimators.append(('log1p', transformer_log1p))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, nb_epoch=20,batch_size=100, verbose=1, validation_data=(testing_X, testing_y), callbacks=[earlyStopping,modelCheck])))
pipeline_keras_nnet = Pipeline(estimators)

In [ ]:
#uczymy model

pipeline_keras_nnet.fit(training_X,training_y)

In [ ]:
#sprawdźmy jakość naszego modelu na danych testowych

pred_nnet= pipeline_keras_nnet.predict(testing[imp_vars])
pred_nnet_proba = pipeline_keras_nnet.predict_proba(testing[imp_vars])

pred_nnet_proba_1 = list(pd.DataFrame(pred_nnet_proba)[1])


#liczymy macierz błędu dla nnet

cf_nnet=confusion_matrix(testing["Target"],pred_nnet)

cf_nnet_pd = pd.DataFrame(cf_nnet)
cf_nnet_pd.columns=["predykcja 0","predykcja 1"]
cf_nnet_pd.index = ["realne 0","realne 1"]
cf_nnet_pd.plot(kind="bar",title="predykcja konwersji wg modelu nnet (Keras/Theano)")

sens = round(cf_nnet[1][1]*1.00 / np.sum(cf_nnet[1]),2)
spec = round(cf_nnet[0][0]*1.00 / np.sum(cf_nnet[0]),2)
aucc = round(roc_auc_score(y_true=testing["Target"],y_score=pred_nnet_proba_1),2)

print("czułość: "+str(sens))
print("specyficzność: "+str(spec))
print("powierzchnia pod krzywą:"+str(aucc))


In [ ]:
fpr_nnet, tpr_nnet, thresholds_nnet = roc_curve(testing["Target"], pred_nnet_proba_1, pos_label=1, drop_intermediate = False)

roc_auc_nnet = auc(fpr_nnet, tpr_nnet)


In [ ]:
plt.figure()
lw = 2

plt.plot(fpr_svm, tpr_svm, color='darkviolet',
         lw=lw, label=u'Krzywa ROC svm (obszar pod krzywą = %0.2f)' % roc_auc_svm, linestyle=':')
plt.plot(fpr_nnet, tpr_nnet, color='darkblue',
         lw=lw, label=u'Krzywa ROC nnet (obszar pod krzywą = %0.2f)' % roc_auc_nnet, linestyle=':')


plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel(u'1 - Specyficzność')
plt.ylabel(u'Czułość')
plt.title('Krzywe ROC dla wielu modeli')
plt.legend(loc="lower right")
plt.show()


## Można skorzystać z kilku modeli na raz :) 

### http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html

In [ ]:
#krok1 - definiujemy modele

clf1 = LogisticRegressionCV(Cs=10, fit_intercept=True, cv=5, dual=False, penalty='l2', scoring=None, solver='lbfgs', tol=0.0001, max_iter=100, class_weight=None, n_jobs=-1, verbose=1, refit=True, intercept_scaling=1.0, multi_class='ovr', random_state=seed)

clf2 = ExtraTreesClassifier(n_estimators=500, criterion='gini', max_depth=10, min_samples_split=20, min_samples_leaf=15, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, bootstrap=True, oob_score=True, n_jobs=-1, random_state=seed, verbose=1, warm_start=False, class_weight=None)

In [ ]:
#krok 2 - korzystamy z VotingClassifier w sklearn

eclf = VotingClassifier(estimators=[('lr', clf1),('xtrees',clf2)],voting='soft')

In [ ]:
estimators_eclf = []
estimators_eclf.append(('log1p', transformer_log1p))
estimators_eclf.append(('eclf', eclf))
pipeline_eclf = Pipeline(estimators_eclf)

In [ ]:
pipeline_eclf.fit(training_X,training_y)

In [ ]:
#sprawdźmy jakość naszego modelu na danych testowych

pred_eclf= pipeline_eclf.predict(testing[imp_vars])
pred_eclf_proba = pipeline_eclf.predict_proba(testing[imp_vars])

pred_eclf_proba_1 = list(pd.DataFrame(pred_eclf_proba)[1])

#liczymy macierz błędu dla ensemble

cf_eclf=confusion_matrix(testing["Target"],pred_eclf)

cf_eclf_pd = pd.DataFrame(cf_eclf)
cf_eclf_pd.columns=["predykcja 0","predykcja 1"]
cf_eclf_pd.index = ["realne 0","realne 1"]
cf_eclf_pd.plot(kind="bar",title="predykcja konwersji wg modelu Ensemble")

sens = round(cf_eclf[1][1]*1.00 / np.sum(cf_eclf[1]),2)
spec = round(cf_eclf[0][0]*1.00 / np.sum(cf_eclf[0]),2)
aucc = round(roc_auc_score(y_true=testing["Target"],y_score=pred_eclf_proba_1),2)

print("czułość: "+str(sens))
print("specyficzność: "+str(spec))
print("powierzchnia pod krzywą:"+str(aucc))


In [ ]:
fpr_eclf, tpr_eclf, thresholds_eclf = roc_curve(testing["Target"], pred_eclf_proba_1, pos_label=1)
roc_auc_eclf = auc(fpr_eclf, tpr_eclf)


In [ ]:
plt.figure()
lw = 2
plt.plot(fpr_nnet, tpr_nnet, color='darkgreen',
         lw=lw, label=u'Krzywa ROC nnet (obszar pod krzywą = %0.2f)' % roc_auc_nnet, linestyle=':')
plt.plot(fpr_xgb, tpr_xgb, color='red',
         lw=lw, label=u'Krzywa ROC xgboost (obszar pod krzywą = %0.2f)' % roc_auc_xgb, linestyle='-.')
plt.plot(fpr_eclf, tpr_eclf, color='darkblue',
         lw=lw, label=u'Krzywa ROC Ensemble (obszar pod krzywą = %0.2f)' % roc_auc_eclf, linestyle='--')


plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel(u'1 - Specyficzność')
plt.ylabel(u'Czułość')
plt.title('Krzywe ROC dla wielu modeli')
plt.legend(loc="lower right")
plt.show()


## Sprawdźmy na koniec korelacje między wynikami

In [ ]:
pred_frame = pd.DataFrame()

pred_frame["ensemble"] = list(pred_eclf_proba_1)
pred_frame["xgboost"] = list(pred_xgb_proba_1)
pred_frame["nnet"] = list(pred_nnet_proba_1)

print("korelacje między modelami predykcyjnymi:")
print(str(pred_frame.corr()))

In [ ]:
sns.pairplot(pred_frame)

### Sprawdźmy dystrybucję realnych 0 i 1 wobec zaokrąglonego prawdopodobieństwa 1 wg modelu xgboost

##### Uwaga: jeśli chcemy znormalizować rozkład prawdopodobieństw nie tracąc dokładności modelu można pomyśleć o regresji izotonicznej: http://scikit-learn.org/stable/modules/generated/sklearn.calibration.CalibratedClassifierCV.html


In [ ]:
pred_stats = pd.DataFrame()

pred_stats["Target"]=testing["Target"]
pred_stats["proba1"]=[round(x,1) for x in pred_xgb_proba_1]
pred_stats["liczba"]=1

pred_stats_show= pred_stats.groupby(['Target', 'proba1'], as_index=False).count()

sns.barplot(x="proba1", y="liczba", hue="Target", data=pred_stats_show)                                  
                                  

### Przy selekcji finalnego modelu możemy też skorzystać z krzywej Precyzja-Czułość (Precision-Recall Curve)

In [ ]:
precision_xgb, recall_xgb, thresholds_xgb = precision_recall_curve(y_true=testing["Target"],probas_pred=pred_xgb_proba_1, pos_label=1)
average_precision_xgb  = average_precision_score(testing["Target"],pred_xgb_proba_1)

precision_nnet, recall_nnet, thresholds_nnet = precision_recall_curve(y_true=testing["Target"],probas_pred= pred_nnet_proba_1, pos_label=1)
average_precision_nnet  = average_precision_score(testing["Target"],pred_nnet_proba_1)

precision_svm, recall_svm, thresholds_svm = precision_recall_curve(y_true=testing["Target"],probas_pred= pred_svm_proba_1, pos_label=1)
average_precision_svm  = average_precision_score(testing["Target"],pred_svm_proba_1)

lw=2
plt.clf()
plt.plot(recall_xgb, precision_xgb, lw=lw, color='red',
         label=u'xgboost (średnia precyzja = %0.2f)' % average_precision_xgb, linestyle=':')
plt.plot(recall_nnet, precision_nnet, lw=lw, color='darkblue',
         label=u'nnet (średnia precyzja = %0.2f)' % average_precision_nnet, linestyle='-')
plt.plot(recall_svm, precision_svm, lw=lw, color='darkgreen',
         label=u'svm (średnia precyzja = %0.2f)' % average_precision_svm, linestyle='--')
plt.xlabel(u'Czułość')
plt.ylabel(u'Precyzja')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.legend(loc="lower left")
plt.title(u"Krzywe Precyzja-Czułość dla wielu modeli")
plt.show()


##  Wdrożenie najlepszych modeli


In [ ]:
#przygotujmy dane

dataset_apply[miss_vars] = dataset_apply[miss_vars].fillna(dataset_apply[miss_vars].median())

apply_data = dataset_apply[imp_vars].as_matrix()

print("dane wdrożeniowe: "+str(apply_data.shape))


In [ ]:
#wdrożenie modelu z keras i ensemle - przetworzenie danych przy użyciu log1p następuje dzięki pipeline

pred_nnet_prob = pipeline_keras_nnet.predict_proba(apply_data)

pred_ensemble_prob = pipeline_eclf.predict_proba(apply_data)

#w przypapadku xgboost musimy dodatkowo użyć funkcji log1p
apply_data_xgboost = dataset_apply[imp_vars].apply(np.log1p, axis=0).as_matrix()

pred_xgboost_prob = model_xgbm.predict_proba(apply_data_xgboost)




In [ ]:
#zbieramy dane do jednego DataFrame i zwizualizujemy

predictions = pd.DataFrame()

predictions["userid"]=dataset_apply["userid"]

predictions["nnet"]=pd.DataFrame(pred_nnet_prob)[1]
predictions["ensemble"]=pd.DataFrame(pred_ensemble_prob)[1]
predictions["xgboost"]=pd.DataFrame(pred_xgboost_prob)[1]

#sprawdźmy dane
predictions.iloc[1:20,:]

In [ ]:
sns.pairplot(predictions.iloc[:,1:4])

In [ ]:
print("korelacje między predykcjami:")
print(str(predictions.iloc[:,1:4].corr()))



In [ ]:
#zapis do pliku csv

predictions.to_csv("results/predictions.csv", index=False, header=True,sep=";")


## Dziękuję za udział w warsztatach, chętnie wysłucham Państwa uwag i pytań :)

### Literatura warta uwagi:

<img src="img/book1.jpg", width=400,height=400>
<img src="img/book2.jpg", width=400,height=400>
<img src="img/book3.png", width=400,height=400>
<img src="img/book4.jpg", width=400,height=400>
<img src="img/book5.jpg", width=400,height=400>